In [1]:
%matplotlib qt5

In [2]:
import mne
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import mne_bids 
import mne_bids.utils 
import pandas as pd

In [3]:
import sys
sys.path.append('/home/lfa-01/Documentos/RepositoriosGit/ripple_IIS_detection/source/')

import SpikeWaveDetectorClass 
import RippleDetectorClass

In [102]:
import importlib
importlib.reload(SpikeWaveDetectorClass)
importlib.reload(RippleDetectorClass)

<module 'RippleDetectorClass' from '/home/lfa-01/Documentos/RepositoriosGit/ripple_IIS_detection/source/RippleDetectorClass.py'>

Se indica la ruta del proyecto base con los datos organizados en formato BIDS

In [4]:
bids_root = '/home/lfa-01/Documentos/Datos_Proyecto_LFA-ENYS_BIDS'
mne_bids.print_dir_tree(bids_root, max_depth=4)

|Datos_Proyecto_LFA-ENYS_BIDS/
|--- README
|--- dataset_description.json
|--- participants.json
|--- participants.tsv
|--- sub-03/
|------ ses-day1/
|--------- sub-03_ses-day1_scans.tsv
|--------- beh/
|--------- ieeg/
|------------ sub-03_ses-day1_task-msl_channels.tsv
|------------ sub-03_ses-day1_task-msl_events.tsv
|------------ sub-03_ses-day1_task-msl_ieeg.edf
|------------ sub-03_ses-day1_task-msl_ieeg.json
|--- sub-04/
|------ ses-day1/
|--------- sub-04_ses-day1_scans.tsv
|--------- beh/
|------------ sub-04_ses-day1_task-mslCtr_desc-results_beh.mat
|------------ sub-04_ses-day1_task-mslExp_desc-results_beh.mat
|------------ sub-04_ses-day1_task-vmaBaseline_beh.mat
|------------ sub-04_ses-day1_task-vmaClamp_beh.mat
|--------- ieeg/
|------------ sub-04_ses-day1_task-msl_channels.tsv
|------------ sub-04_ses-day1_task-msl_events.tsv
|------------ sub-04_ses-day1_task-msl_ieeg.edf
|------------ sub-04_ses-day1_task-msl_ieeg.json
|------------ sub-04_ses-day1_task-vma_channels.t

Se indica informacion especifica para ubicar a un sujeto y tarea particular

In [5]:
session = 'day1'
datatype = 'ieeg'
subjects = mne_bids.get_entity_vals(bids_root, 'subject')
tasks = mne_bids.get_entity_vals(bids_root, 'task')
suffix = 'ieeg'

print(mne_bids.get_entity_vals(bids_root, 'subject'))
print(mne_bids.get_entity_vals(bids_root, 'task'))
print(mne_bids.get_entity_vals(bids_root, 'session'))
print(mne_bids.get_entity_vals(bids_root, 'run'))

['03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
['msl', 'mslCtr', 'mslExp', 'vma', 'vmaBaseline', 'vmaClamp']
['day1']
[]


Creacion del BIDS path

In [6]:
idx_sub = 0 # Indice del sujeto analizado
idx_task = 0 # Indice de la tarea

bids_path = mne_bids.BIDSPath(root=bids_root, subject = subjects[idx_sub], task = tasks[idx_task], session=session, suffix=suffix, datatype=datatype)
print(bids_path.match(ignore_json=True))

[BIDSPath(
root: /home/lfa-01/Documentos/Datos_Proyecto_LFA-ENYS_BIDS
datatype: ieeg
basename: sub-03_ses-day1_task-msl_ieeg.edf)]


Path derivatives

In [7]:
deriv_bids_root = '/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS'
mne_bids.print_dir_tree(deriv_bids_root, max_depth=4)

bids_path = mne_bids.BIDSPath(root=bids_root, subject = subjects[idx_sub], task = tasks[idx_task], session=session, suffix=suffix, datatype=datatype)
print(bids_path.match(ignore_json=True))

|Derivatives_Proyecto_LFA-ENYS_BIDS/
|--- README
|--- Referencias_guia_tener_en_cuenta.odt
|--- dataset_description.json
|--- participants.json
|--- participants.tsv
|--- sub-03/
|------ ses-day1/
|--------- sub-03_ses-day1_scans.tsv
|--------- PSD_epochs/
|------------ sub-03_ses-day1_desc-ctrTask_spectrumPSD_epo-epoch.npz
|------------ sub-03_ses-day1_desc-ctrTask_spectrumPSD_epo-rest.npz
|------------ sub-03_ses-day1_desc-ctrTask_spectrumPSD_epo-task.npz
|------------ sub-03_ses-day1_desc-secTask_spectrumPSD_epo-epoch.npz
|------------ sub-03_ses-day1_desc-secTask_spectrumPSD_epo-rest.npz
|------------ sub-03_ses-day1_desc-secTask_spectrumPSD_epo-task.npz
|--------- ieeg/
|------------ sub-03_ses-day1_selectedChannels.tsv
|------------ sub-03_ses-day1_task-msl_desc-baselineSegm1_channels.tsv
|------------ sub-03_ses-day1_task-msl_desc-baselineSegm1_events.tsv
|------------ sub-03_ses-day1_task-msl_desc-baselineSegm1_ieeg.edf
|------------ sub-03_ses-day1_task-msl_desc-baselineSegm1_

In [8]:
# Se identifica el archivo a levantar
deriv_bids_path = mne_bids.BIDSPath(root=deriv_bids_root, subject = subjects[idx_sub], task = tasks[idx_task], 
                                    session=session, suffix=suffix, datatype=datatype)
print(deriv_bids_path.match(ignore_json=True))

[BIDSPath(
root: /home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS
datatype: ieeg
basename: sub-03_ses-day1_task-msl_desc-baselineSegm1_ieeg.edf), BIDSPath(
root: /home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS
datatype: ieeg
basename: sub-03_ses-day1_task-msl_desc-baselineSegm2_ieeg.edf), BIDSPath(
root: /home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS
datatype: ieeg
basename: sub-03_ses-day1_task-msl_desc-baselineSegm3_ieeg.edf), BIDSPath(
root: /home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS
datatype: ieeg
basename: sub-03_ses-day1_task-msl_desc-ctrTask_ieeg.edf), BIDSPath(
root: /home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS
datatype: ieeg
basename: sub-03_ses-day1_task-msl_desc-secTask_ieeg.edf)]


Se lee el archivo .edf especificado como un objeto MNE

In [17]:
raw_mne = mne_bids.read_raw_bids(bids_path=bids_path, verbose=False)
srate = raw_mne.info['sfreq']

/tmp/ipykernel_115163/3246288057.py:1: RuntimeWarning: Did not find any electrodes.tsv associated with sub-03_ses-day1_task-msl.

The search_str was "/home/lfa-01/Documentos/Datos_Proyecto_LFA-ENYS_BIDS/sub-03/**/ieeg/sub-03_ses-day1*electrodes.tsv"
  raw_mne = mne_bids.read_raw_bids(bids_path=bids_path, verbose=False)
/tmp/ipykernel_115163/3246288057.py:1: RuntimeWarning: Did not find any coordsystem.json associated with sub-03_ses-day1_task-msl.

The search_str was "/home/lfa-01/Documentos/Datos_Proyecto_LFA-ENYS_BIDS/sub-03/**/ieeg/sub-03_ses-day1*coordsystem.json"
  raw_mne = mne_bids.read_raw_bids(bids_path=bids_path, verbose=False)


Se levanta lista de canales de interes, en particular lo ubicados en hipocampo/s

In [24]:
# Se levantan los canales a utilizar en el registro correspondiente
select_chann_fname = str(deriv_bids_path.directory) + '/sub-' + subjects[idx_sub] + '_ses-' + session + '_selectedChannels' + '.tsv'
print(select_chann_fname)
chann_select_aux = pd.read_csv(select_chann_fname, delimiter='\t')
chann_select = chann_select_aux['name'].to_list()
print(chann_select)

/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-03/ses-day1/ieeg/sub-03_ses-day1_selectedChannels.tsv
['HCD1', 'HCD2', 'HQD1', 'HQD2', 'HCI1', 'HCI2', 'HQI1']


In [30]:
raw_mne_select_channs = raw_mne.copy().pick(chann_select)
print(raw_mne_select_channs.info)

<Info | 8 non-empty values
 bads: []
 ch_names: HCD1, HCD2, HQD1, HQD2, HCI1, HCI2, HQI1
 chs: 7 sEEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: 2023-04-24 13:16:42 UTC
 nchan: 7
 projs: []
 sfreq: 2000.0 Hz
 subject_info: 5 items (dict)
>


Prueba para ver como puedo hacer re-referenciacion bipolar entre canales 

In [31]:
raw_mne_select_channs.load_data()
raw_mne_select_channs_rref = mne.set_bipolar_reference(raw_mne_select_channs, ['HCD1'],['HCD2'],drop_refs=False)

sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=4628000
    Range : 0 ... 4627999 =      0.000 ...  2313.999 secs
Ready.
Added the following bipolar channels:
HCD1-HCD2


In [32]:
print(raw_mne_select_channs_rref.info)
raw_mne_select_channs_rref.info['ch_names']

<Info | 10 non-empty values
 bads: []
 ch_names: HCD1, HCD2, HQD1, HQD2, HCI1, HCI2, HQI1, HCD1-HCD2
 chs: 8 sEEG
 custom_ref_applied: True
 dig: 0 items
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: 2023-04-24 13:16:42 UTC
 nchan: 8
 projs: []
 sfreq: 2000.0 Hz
 subject_info: 5 items (dict)
>


['HCD1', 'HCD2', 'HQD1', 'HQD2', 'HCI1', 'HCI2', 'HQI1', 'HCD1-HCD2']

In [33]:
raw_mne_select_channs_rref.plot(n_channels=10, scalings=dict(eeg=20e-5), start=0, duration=10, block=True)

Using qt as 2D backend.
Channels marked as bad:
none


## Identificacion de IISs (Inter-Ictal Spikes)

In [97]:
# channel index from selected channels
chann_idxs = 9

raw_mne_selected_channel = raw_mne.copy().pick(chann_select[chann_idxs])
print(raw_mne_selected_channel.info)

raw_mne_selected_channel.plot(n_channels=10, scalings=dict(eeg=20e-5), start=0, duration=10, block=True)


<Info | 8 non-empty values
 bads: []
 ch_names: HQUI1
 chs: 1 sEEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: 2022-12-28 13:19:52 UTC
 nchan: 1
 projs: []
 sfreq: 2000.0 Hz
 subject_info: 4 items (dict)
>
Channels marked as bad:
none


### Antes de la detección hay que aplicarle el filtrado notch y pasabanda a los datos raw para eliminar ruido de linea y acotar en banda

In [98]:
notch_freqs = [50, 100, 150, 200, 250]
PB_min = 0.01
PB_max = 300
filt_mne_selected_channel = raw_mne_selected_channel.load_data().copy().notch_filter(freqs=notch_freqs)
filt_mne_selected_channel.filter(l_freq=PB_min, h_freq=PB_max)

Reading 0 ... 3833999  =      0.000 ...  1916.999 secs...


Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 13201 samples (6.601 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 3e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.01
- Lower transition bandwidth: 0.01 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 300.00 Hz
- Upper transition bandwidth: 75.00 Hz (-6 dB cutoff frequency: 337.50 Hz)
- Filter length: 660001 samples (330.000 s)



<RawEDF | sub-07_ses-day1_task-msl_ieeg.edf, 1 x 3834000 (1917.0 s), ~29.3 MB, data loaded>

In [99]:
filt_mne_selected_channel.plot(n_channels=10, scalings=dict(eeg=20e-5), start=0, duration=10, block=True)

Channels marked as bad:
none


Extract filtered data

In [100]:
seeg_data_single_chann, times = filt_mne_selected_channel[:, :]

## Identificación de Inter-Ictal spikes

In [101]:
# Instantiate SpikeWaveDetectorClass
IIS_det = SpikeWaveDetectorClass.SpikeWaveDetectorClass(int(srate))

In [102]:
peak_times, peak_stats = IIS_det.detect_times(seeg_data_single_chann, True)

In [103]:
print(len(peak_times))
print(peak_times)

194
[   9112.   15887.   16095.   16298.   16801.   16954.   80430.   80926.
  168978.  169159.  218415.  238116.  328004.  328336.  342646.  342851.
  371623.  372633.  374046.  438270.  440291.  480455.  527825.  566044.
  574457.  576020.  576191.  576305.  576430.  636862.  640109.  747290.
  761565.  785701.  792290.  880682.  907149.  924032. 1065352. 1094384.
 1102087. 1169780. 1195551. 1196158. 1196323. 1201651. 1213272. 1231281.
 1248033. 1371206. 1401205. 1517489. 1555176. 1584302. 1584507. 1584775.
 1584918. 1585226. 1585698. 1587075. 1601402. 1603948. 1645222. 1668712.
 1688877. 1729458. 1736054. 1762526. 1762674. 1762923. 1784292. 1834770.
 1873735. 1875071. 1876013. 1881470. 1884415. 1888510. 1950718. 1988194.
 1988538. 1988659. 1988862. 1990394. 2042136. 2069311. 2097672. 2099748.
 2100108. 2100216. 2100671. 2101013. 2101978. 2113424. 2174162. 2174619.
 2175079. 2321780. 2330585. 2340119. 2340376. 2345791. 2350886. 2363260.
 2363919. 2364645. 2400358. 2411092. 2427996. 2

In [46]:
keys = peak_stats.keys()
print(keys)

print(len(peak_stats['zscores_per_peaks_max']))
print(len(peak_stats['zscores_per_peaks_env']))
print(len(peak_stats['zscores_per_peaks_amp']))
print(len(peak_stats['zscores_per_peaks_grad']))
print(len(peak_stats['inds_per_peak']))
print(len(peak_stats['passed_conditions']))

print(type(peak_stats['zscores_per_peaks_max']))
print(type(peak_stats['zscores_per_peaks_env']))
print(type(peak_stats['zscores_per_peaks_amp']))
print(type(peak_stats['zscores_per_peaks_grad']))
print(type(peak_stats['inds_per_peak']))
print(type(peak_stats['passed_conditions']))

dict_keys(['zscores_per_peaks_max', 'zscores_per_peaks_env', 'zscores_per_peaks_amp', 'zscores_per_peaks_grad', 'inds_per_peak', 'passed_conditions'])
96
96
96
96
96
96
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [33]:
plot_z_scores = True
IIS_det.plot_spike_waves(seeg_data_single_chann, peak_times, 1, peak_stats, plot_z_scores)  

## Deteccion de Ripples

In [104]:
# Instantiate RippleDetectorClass
Ripp_det = RippleDetectorClass.RippleDetectorClass(int(srate))

In [105]:
ripple_times, ripple_start_end = Ripp_det.detect_ripple(seeg_data_single_chann, None, peak_times, None)

In [106]:
len(ripple_times)

145

In [ ]:
Ripp_det.plot_ripples(seeg_data_single_chann, ripple_times, None)

Distribución de la duración de los ripples posible detectados para ver que tenga sentido 

In [107]:
ripple_len = np.diff(np.array(ripple_start_end)) / filt_mne_selected_channel.info['sfreq']
plt.hist(ripple_len, bins=40)
plt.xlabel('Ripple Length [seg]')
plt.ylabel('Counts')
plt.title('Histogram of Ripple Length')
plt.show()


Construccion de matriz de eventos para visualizar junto con la señal

In [108]:
events_array = np.column_stack((np.concatenate((ripple_times, peak_times)), np.zeros(len(ripple_times)+len(peak_times)), np.concatenate((np.ones(len(ripple_times)),2*np.ones(len(peak_times))))))

events_array = events_array.astype(int)
print(events_array)

events_id = {'putative_ripple': 1, 'putative_IIS': 2}
event_color = {'putative_ripple': 'r', 'putative_IIS': 'k'}

[[   1035       0       1]
 [   3437       0       1]
 [  10942       0       1]
 ...
 [3666647       0       2]
 [3731824       0       2]
 [3735588       0       2]]


Plot de la señal con eventos indicando posicles IIS y ripples

In [109]:
filt_mne_selected_channel.plot(events=events_array, event_id=events_id, event_color=event_color, duration=15,
         remove_dc=False, scalings = 'auto', block=True, color=dict(seeg='b'))

Channels marked as bad:
none


### Identificación de falsos positivos siguiendo las recomendaciones de Ngo et al., 2020, eLife

Para ello:
* se toman epocas de +-500ms
* se computa tiempo-frecuencia, que se hizo con mutitapers en este caso, en el rango de 65 a 130 Hz (de a 2 Hz), y en toda la ventana temporal cada 2 ms
* luego se hace un promedio a lo largo del tiempo entre los +-50ms y se conserva solo el perfil resultante en el rango de frecuencias de interes para los ripples, de 80 a 120 Hz
* Mirando el perfil de potencia para cada epoca, se determina el maximo de la potencia promedio y un umbral del 80% de este valor. Si al inicio y al final del perfil hay un descenso por debajo del umbral, se considera un ripples valido. Esto es fundamentalmente porque implica que hay un aumento de la porencia concentrado en ese rango de frecuencias. 

Epocas

In [110]:
epochs = mne.Epochs(filt_mne_selected_channel, events_array, tmin=-0.5, tmax=0.5, event_id=events_id, preload=True)
#epochs = mne.Epochs(raw_mne_selected_channel, events_array, tmin=-0.5, tmax=0.5, event_id=events_id, preload=True)


Not setting metadata
339 matching events found
Setting baseline interval to [-0.5, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 339 events and 2001 original time points ...
0 bad epochs dropped


/tmp/ipykernel_828458/1727979436.py:1: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs = mne.Epochs(filt_mne_selected_channel, events_array, tmin=-0.5, tmax=0.5, event_id=events_id, preload=True)


Tiempo-frecuencia con multitapers

In [111]:
min_freq = 65
max_freq = 135
freqs = np.arange(min_freq, max_freq+2, 2)  # 
n_cycles = freqs/500
t_win = n_cycles / freqs
#freq_bins = int((max_freq - min_freq)/2)
#freqs = np.logspace(*np.log10([min_freq, max_freq]), num=freq_bins)
print(freqs)
print(t_win)

[ 65  67  69  71  73  75  77  79  81  83  85  87  89  91  93  95  97  99
 101 103 105 107 109 111 113 115 117 119 121 123 125 127 129 131 133 135]
[0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002]


In [112]:
power = mne.time_frequency.tfr_multitaper(
    epochs,
    freqs = freqs,
    n_cycles = freqs,
    use_fft = True,
    return_itc = False,
    average = False,
    decim = 2,
)

Not setting metadata


Identificacion de un perfil de potencia acorde con los ripples

In [113]:
def is_prominent_peak(signal, peak_index):
    peak_amplitude = signal[peak_index]
    threshold_amplitude = 0.8 * peak_amplitude
    
    # Check if amplitude at the beginning and end is lower than threshold
    beginning_amplitude = signal[0]
    end_amplitude = signal[-1]
    
    return beginning_amplitude <= threshold_amplitude and end_amplitude <= threshold_amplitude

In [114]:
freq_profile = []
valid_ripple = np.zeros(len(power['putative_ripple']))

for epoch_trf_idx in range(len(power['putative_ripple'])): 
    #epoch_trf_idx = 0 # index opf the epoch time-freq representation
    
    ch_idx = 0 # channel index
    #print(power['putative_ripple'][epoch_trf_idx].data[:,ch_idx][0].shape)

    data_aux = power['putative_ripple'][epoch_trf_idx].data[:,ch_idx][0]
    # me quedo con los datos en la ventana de +-50ms
    data_aux = data_aux[:, int(data_aux.shape[1]/2)-50:int(data_aux.shape[1]/2)+50]
    #print(data_aux.shape)
    data_aux_mean = np.mean(data_aux, axis=1)
    #print(data_aux_mean.shape)

    # se conservan solo las frecuencias de interes
    freqs_interest_idx = np.where((freqs >= 80) & (freqs <= 120))
    data_aux_mean = data_aux_mean[freqs_interest_idx]
    #print(data_aux_mean.shape)

    # concatenate freq_profile with data_aux_mean
    freq_profile.append(data_aux_mean)
    

    '''plt.plot(freqs[freqs_interest_idx], data_aux_mean)
    plt.xlabel('Frequency')
    plt.ylabel('Mean Data')
    plt.title('Mean Data vs Frequency')
    plt.show()'''

    # Se determina el maximo de la potencia promedio dentro de las frecuencias de interés
    # y el umbral del 80% de este valor
    '''max_value = np.max(data_aux_mean)
    threshold = 0.8 * max_value  # 20% del valor máximo


    if data_aux_mean[0] <= threshold and data_aux_mean[-1] <= threshold:
        #print("Los primer y último puntos muestran una caída de al menos el 20% respecto al valor máximo.")
        valid_ripple[epoch_trf_idx] = 1
    #else:
        #print("Los primer y último puntos no muestran una caída de al menos el 20% respecto al valor máximo.")
    '''
    # VARIANTE
    peaks, _ = signal.find_peaks(data_aux_mean)
    if len(peaks) == 0:
        valid_ripple[epoch_trf_idx] = 0
    else:
        # Get the index of the highest peak
        prominent_peak_index = np.argmax(data_aux_mean[peaks])

        # Check if the prominent peak is prominent enough
        if is_prominent_peak(data_aux_mean, peaks[prominent_peak_index]):
            valid_ripple[epoch_trf_idx] = 1
        else:
            valid_ripple[epoch_trf_idx] = 0


freq_profile = np.array(freq_profile)
print(freq_profile.shape)
print(np.sum(valid_ripple))
print(valid_ripple)


(145, 20)
94.0
[1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1.
 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1.
 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0.
 0.]


Gráfico de todos los perfiles de potencia como control

In [115]:
# Plot all rows of freq_profile
freq_profile_valid = freq_profile[valid_ripple == 1]
for row in freq_profile_valid:
    plt.plot(freqs[freqs_interest_idx], row, alpha=0.2)

# Set labels and title
plt.xlabel('Frequency')
plt.ylabel('Mean Data')
plt.title('Mean Data vs Frequency')

# Show the plot
plt.show()

In [116]:
epochs['putative_ripple'][valid_ripple == 1].plot_image(combine="mean")

Not setting metadata
94 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_828458/655699615.py:1: RuntimeWarning: Only one channel in group "seeg"; cannot combine by method "mean".
  epochs['putative_ripple'][valid_ripple == 1].plot_image(combine="mean")


combining channels using "mean"


/tmp/ipykernel_828458/655699615.py:1: RuntimeWarning: Only 1 channel in "picks"; cannot combine by method "mean".
  epochs['putative_ripple'][valid_ripple == 1].plot_image(combine="mean")


[<Figure size 640x480 with 3 Axes>]

Construccion de array de eventos filtrado por los supuestos ripples validos

In [117]:
events_array_valid = np.column_stack((np.concatenate(([ripple_times[i] for i in np.where(valid_ripple)[0]], peak_times)), 
                                      np.zeros(len([ripple_times[i] for i in np.where(valid_ripple)[0]])+len(peak_times)), 
                                      np.concatenate((np.ones(len([ripple_times[i] for i in np.where(valid_ripple)[0]])),
                                                      2*np.ones(len(peak_times))))))

events_array_valid = events_array_valid.astype(int)
print(events_array_valid)

events_id = {'putative_ripple': 1, 'putative_IIS': 2}
event_color = {'putative_ripple': 'r', 'putative_IIS': 'k'}

[[   1035       0       1]
 [   3437       0       1]
 [  10942       0       1]
 [  12181       0       1]
 [  13850       0       1]
 [  19464       0       1]
 [  36488       0       1]
 [  73310       0       1]
 [  76204       0       1]
 [  76819       0       1]
 [  77557       0       1]
 [  79289       0       1]
 [  85815       0       1]
 [  86521       0       1]
 [ 110057       0       1]
 [ 110208       0       1]
 [ 110889       0       1]
 [ 111255       0       1]
 [ 111794       0       1]
 [ 113616       0       1]
 [ 143492       0       1]
 [ 167698       0       1]
 [ 329587       0       1]
 [ 330150       0       1]
 [ 824073       0       1]
 [ 824545       0       1]
 [ 824637       0       1]
 [ 826036       0       1]
 [ 826412       0       1]
 [ 827799       0       1]
 [ 828790       0       1]
 [ 854118       0       1]
 [ 865319       0       1]
 [ 874962       0       1]
 [ 875537       0       1]
 [ 876602       0       1]
 [ 884040       0       1]
 

In [92]:
filt_mne_selected_channel.plot(events=events_array_valid, event_id=events_id, event_color=event_color, duration=15,
         remove_dc=False, scalings = 'auto', block=True, color=dict(seeg='b'))

Channels marked as bad:
none


Potencia promedio incluyendo solo las epocas validas

In [118]:
power['putative_ripple'][valid_ripple == 1].average().plot()
#power['putative_ripple'][0].plot()

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
No baseline correction applied


[<Figure size 640x480 with 2 Axes>]

Aca se realiza un nuevo analisis tiempo frecuencia, pero ahora con waveletes y normalizando respecto a un baseline arbitrario para verificar que se observa una representación consistente con la existencia de ripples

In [119]:
freqs_wvlt = np.logspace(*np.log10([60, 160]), num=30)
n_cycles_wvlt = freqs_wvlt / 2.0  # different number of cycle per frequency
power_wvlt, itc_wvlt = mne.time_frequency.tfr_morlet(
    epochs['putative_ripple'][valid_ripple == 1], # filtrada de acuerdo a ripples validos
    freqs=freqs_wvlt,
    n_cycles=n_cycles_wvlt,
    use_fft=True,
    return_itc=True,
    decim=3,
    n_jobs=None,
)

In [120]:
#power_wvlt.plot()
power_wvlt.plot(baseline=(-0.5, -0.2), mode="logratio")

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Applying baseline correction (mode: logratio)


[<Figure size 640x480 with 2 Axes>]

### Visualizacion de las epocas en el dominio temporal (stack y promedio)

In [96]:
# Plot all rows of freq_profile [900:1100]
for n_epoch in range(len(epochs['putative_ripple'][valid_ripple == 1])):
    plt.plot(epochs['putative_ripple'][valid_ripple == 1].times,
             epochs['putative_ripple'][valid_ripple == 1][n_epoch].get_data()[0][0], color='lightgrey', linewidth=0.5)

plt.plot(epochs['putative_ripple'][valid_ripple == 1].times,
         epochs['putative_ripple'][valid_ripple == 1][:].get_data().mean(axis=0)[0], color='black', linewidth=1)


# Show the plot
plt.show()

#print(epochs_filt['putative_ripple'][valid_ripple == 1][0].get_data()[0])#

In [50]:
print(epochs['putative_ripple'][valid_ripple == 1][:].get_data().mean(axis=0)[0].shape)

(2001,)


In [43]:
print(epochs['putative_ripple'][valid_ripple == 1][0].times)

[-0.5    -0.4995 -0.499  ...  0.499   0.4995  0.5   ]


In [40]:
epochs['putative_ripple'][valid_ripple == 1].plot()

/tmp/ipykernel_203892/165650962.py:1: FutureWarning: The current default events=None is deprecated and will change to events=True in MNE 1.6. Set events=False to suppress this warning.
  epochs['putative_ripple'][valid_ripple == 1].plot()


Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none


In [10]:
import numpy as np
from scipy import signal

# Create a triangular signal
samples = 30
max_value = 20
triangular_signal = signal.sawtooth(np.linspace(0, 10, samples), width=1) * max_value

# Print the triangular signal
print(triangular_signal)

[-20.         -17.80475941 -15.60951881 -13.41427822 -11.21903762
  -9.02379703  -6.82855643  -4.63331584  -2.43807525  -0.24283465
   1.95240594   4.14764654   6.34288713   8.53812773  10.73336832
  12.92860892  15.12384951  17.3190901   19.5143307  -18.29042871
 -16.09518811 -13.89994752 -11.70470692  -9.50946633  -7.31422574
  -5.11898514  -2.92374455  -0.72850395   1.46673664   3.66197724]
